In [1]:
import sys, os
sys.path.append('../')
from xd import *
from utils import *
import esutil
import healpy as hp
from systematics import *
from matplotlib.backends.backend_pdf import PdfPages


from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
## for Palatino and other serif fonts use:
#rc('font',**{'family':'serif','serif':['Palatino']})
rc('text', usetex=True)

In /users/PCON0003/warner785/.conda/envs/DES_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /users/PCON0003/warner785/.conda/envs/DES_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /users/PCON0003/warner785/.conda/envs/DES_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


In [4]:
def valid_hp(cat, nside = 4096):
    
    hpind = hpRaDecToHEALPixel(cat['RA'], cat['DEC'], nside=nside, nest= False)
    hpind_reject = hpRaDecToHEALPixel(cat['RA'][cat['VETO_FLAG'] == 0], 
                                    cat['DEC'][cat['VETO_FLAG'] == 0], nside=nside, nest= False)
    
    hppixnumber = np.arange(hp.nside2npix(nside))
    
    geomask = np.in1d( hppixnumber, hpind )
    rejectmask = np.in1d( hppixnumber, hpind_reject )
    
    print(np.sum(geomask), np.sum(~rejectmask))
    
    
    totalmask = geomask * (~rejectmask)
    return hppixnumber[totalmask]


def hp_mask(cat, valid_hpind, nside = 4096):
    hpind = hpRaDecToHEALPixel(cat['RA'], cat['DEC'], nside=nside, nest= False)
    HPmask = np.in1d(hpind, valid_hpind)
    return HPmask




def callingBOSSmask(option='SGC'):
    
    if option == 'SGC':
        valid_hpind = fitsio.read('/n/des/lee.5922/data/cmass_cat/vetomask/mask_nside4096_Patchy-Mocks-Randoms-DR12SGC-COMPSAM_V6C.fits')
    elif option == 'NGC': 
        valid_hpind = fitsio.read('/n/des/lee.5922/data/cmass_cat/vetomask/mask_nside4096_Patchy-Mocks-Randoms-DR12NGC-COMPSAM_V6C.fits')

    CleanMask = np.zeros((valid_hpind.size, ), dtype=[('PIXEL', 'i4'),('RA', 'f8'), ('DEC', 'f8')])
    CleanMask['PIXEL'] = valid_hpind
    sys_ra, sys_dec = hpHEALPixelToRaDec(valid_hpind, nside = 4096)
    CleanMask['RA'] = sys_ra
    CleanMask['DEC'] = sys_dec
    return CleanMask

def BOSSvetoing(cat, option='SGC'):
    
    if option == 'SGC':
        valid_hpind = esutil.io.read('/n/des/lee.5922/data/cmass_cat/vetomask/mask_nside4096_Patchy-Mocks-Randoms-DR12SGC-COMPSAM_V6C.fits')
    elif option == 'NGC': 
        valid_hpind = esutil.io.read('/n/des/lee.5922/data/cmass_cat/vetomask/mask_nside4096_Patchy-Mocks-Randoms-DR12NGC-COMPSAM_V6C.fits')
    elif option =='all':
        valid_hpind = esutil.io.read(['/n/des/lee.5922/data/cmass_cat/vetomask/mask_nside4096_Patchy-Mocks-Randoms-DR12NGC-COMPSAM_V6C.fits',
                                   '/n/des/lee.5922/data/cmass_cat/vetomask/mask_nside4096_Patchy-Mocks-Randoms-DR12SGC-COMPSAM_V6C.fits'],
                                 combine=True)
    hpind = hpRaDecToHEALPixel(cat['RA'], cat['DEC'], nside=4096, nest= False)
    HPmask = np.in1d(hpind, valid_hpind)
    
    return cat[HPmask]

def keepGoodRegion(des, hpInd = False, balrog=None):
    import healpy as hp
    import fitsio
    
    path = '/fs/scratch/PCON0008/warner785/bwarner/'
    LSSGoldmask = fitsio.read(path+'MASK_Y3LSSBAOSOF_22_3_v2p2.fits')
    ringhp = hp.nest2ring(4096, [LSSGoldmask['PIXEL']])
    ind_good_ring = ringhp
    
    # healpixify the catalog.
    nside=4096
    # Convert silly ra/dec to silly HP angular coordinates.
    phi = des['RA'] * np.pi / 180.0
    theta = ( 90.0 - des['DEC'] ) * np.pi/180.0

    hpInd = hp.ang2pix(nside,theta,phi,nest=False)
    keep = np.in1d(hpInd, ind_good_ring)
    des = des[keep]
    if hpInd is True:
        return ind_good_ring
    else:
        return des

In [ ]:
#hpix_dmass = hpRaDecToHEALPixel(dmass_spt['RA'], dmass_spt['DEC'], nside=4096, nest= False)
#dmass_spt_hpix = appendColumn(dmass_spt, name = 'PIXEL', value = hpix_dmass, dtypes=None)

from run_systematics import sys_iteration, fitting_SP, sys_ngal, weightmultiply, fitting_allSP, calling_sysMap, maskingCatalogSP

sysMap = calling_sysMap( properties=['FWHM'], kind='SPT', nside=4096 )

cmass_mock_fileN = np.loadtxt(path+'patchy_mock/Patchy-Mocks-DR12CMASS-N-V6C-Portsmouth-mass_0011.dat')
cmass_mock_fileS = np.loadtxt(path+'patchy_mock/Patchy-Mocks-DR12CMASS-S-V6C-Portsmouth-mass_0011.dat')
cmass_mock_file = np.vstack([cmass_mock_fileN, cmass_mock_fileS])

#cmass_mock_file = np.loadtxt(path+'patchy_mock/Patchy-Mocks-DR12CMASS-N-V6C-Portsmouth-mass_0001.dat')
cmass_mock = np.zeros( (cmass_mock_file.shape[0],), dtype = [('RA', 'float'), ('DEC', 'float'), ('Z', 'float'),('VETO_FLAG', 'int')])
cmass_mock['RA'] = cmass_mock_file[:,0]
cmass_mock['DEC'] = cmass_mock_file[:,1]
cmass_mock['Z'] = cmass_mock_file[:,2]
cmass_mock['VETO_FLAG'] = cmass_mock_file[:,6]
#cmass_mock = cmass_mock[(cmass_mock['Z'] > 0.43) & (cmass_mock['Z'] < 0.7)]
cmass_mock = cmass_mock[ cmass_mock['VETO_FLAG'] == 1]

cmass_mock = BOSSvetoing(cmass_mock, option = 'all')

cmass_mock_fileN, cmass_mock_fileS, cmass_mock_file = None, None, None

In [ ]:
BOSSCleanMask = np.hstack([BOSSCleanMaskSGC['PIXEL'], BOSSCleanMaskNGC['PIXEL']])
Narr = BOSSCleanMask.size/615362 #600000
HIND_BOSSCleanMask = np.split( BOSSCleanMask[:615362*Narr], Narr)
#HIND_CMASSCleanMask = np.array_split( BOSSCleanMask, BOSSCleanMask.size/600000)


n_density_CMASS = np.zeros(len(HIND_CMASSCleanMask))
i = 0
for vind in HIND_CMASSCleanMask:
    print '\r', i, '/',len(HIND_CMASSCleanMask),
    n_density_CMASS[i] = CalculateNumDensityArea(cmass, vind, weight = True )
    i += 1

    
avg_n_density_cmass = n_density_CMASS * 1./n_density_CMASS.mean()
mean_n_density_cmass = np.mean(avg_n_density_cmass)
std_n_density_cmass = np.std(avg_n_density_cmass)
area_cmass = hp.nside2pixarea( 4096, degrees = True) * HIND_CMASSCleanMask[0].size
print 'Area', area_cmass

In [ ]:
sys_mask = (sysMap['sys_FWHM_r_SPT']['SIGNAL'] < 4.5)
map_mask = np.in1d( GoldMask_spt['PIXEL'],  sysMap['sys_FWHM_r_SPT']['PIXEL'][sys_mask])
GoldMask_spt_sysmasked = GoldMask_spt[map_mask]
sptnpix_sys = np.sum(GoldMask_spt_sysmasked['FRAC']) 
#print np.sum(sys_mask), np.sum(map_mask)
#print GoldMask_spt.size, GoldMask_spt_sysmasked.size
SPTMaparea_sys = pixarea * sptnpix_sys
print SPTMaparea_sys, SPTMaparea

import numpy as np

path = '/fs/scratch/PCON0008/warner785/bwarner/'

cmass_ngc = esutil.io.read(path+'galaxy_DR12v5_CMASS_North.fits.gz')
cmass_sgc = esutil.io.read(path+'galaxy_DR12v5_CMASS_South.fits.gz')
#cmass_ngc = esutil.io.read(path+'cmass-dr12v4-N-Reid-full.dat.fits')
#cmass_sgc = esutil.io.read(path+'cmass-dr12v4-S-Reid-full.dat.fits')
#cmass_ngc = cmass_ngc[(cmass_ngc['Z'] > 0.43) & (cmass_ngc['Z'] < 0.7)]
#cmass_sgc = cmass_sgc[(cmass_sgc['Z'] > 0.43) & (cmass_sgc['Z'] < 0.7)]
#cmass_train = Cuts.SpatialCuts(cmass_sgc, ra=310, ra2=360, dec=-2, dec2=2)
cmass_train = keepGoodRegion(cmass_sgc)

cmass = np.hstack([cmass_ngc, cmass_sgc])

print np.sum(cmass_train['WEIGHT_CP'])

In [ ]:
rootdir = '../output/sfd_train_photo_SamPle_nozband_ran10_3_full_tol1e5/'
dmass_spt = io.SearchAndCallFits(path = rootdir, columns = None, keyword = 'dmass_spt_sys_v3.fits')
dmass_spt = dmass_spt[ dmass_spt['CMASS_PROB'] > 0.01 ]
dmass_st82 = io.SearchAndCallFits(path = rootdir, columns = None, keyword = 'gold_st82_prob.fits')
dmass_st82 =dmass_st82[ dmass_st82['EachProb_CMASS'] > 0.01]

train = esutil.io.read(rootdir+'train_sample_des.fits')
train = Cuts.keepGoodRegion(train)

dmass_spt = Cuts.keepGoodRegion(dmass_spt)
dmass_st82 = Cuts.keepGoodRegion(dmass_st82)

print np.sum(dmass_st82['EachProb_CMASS'])
print np.sum(dmass_spt['CMASS_PROB'])
print train.size

import pandas as pd
desData = pd.DataFrame(dmass_spt)
goldData = pd.DataFrame(GoldMask_spt)
matched = pd.merge(desData, goldData, left_on='HPIX', right_on='PIXEL', how='left')
matched_arr = matched.to_records(index=False)

from utils import mergeCatalogsUsingPandas
dmass_spt_hpix = mergeCatalogsUsingPandas(des=dmass_spt, gold=GoldMask_spt, how = 'right', 
                                      left_key='HPIX', right_key = 'PIXEL', suffixes = ['','_GOLD'], 
                                      left_index=False, right_index = False)

BOSSCleanMaskSGC = callingBOSSmask(option='SGC')
BOSSCleanMaskNGC = callingBOSSmask(option='NGC')
BOSSSGCMaparea = BOSSCleanMaskSGC.size * pixarea
BOSSNGCMaparea = BOSSCleanMaskNGC.size * pixarea
print BOSSSGCMaparea, BOSSNGCMaparea

In [ ]:
# NOT USED IN PLOT
# add jack knife error
def jkerror( catalog, random, njack = 10 ):
    import os
    #from suchyta_utils import _jkfunctions as _jk

    # jk error
    jkfile = './jkregion.txt'
    jk.GenerateJKRegions( catalog['RA'], catalog['DEC'], njack, jkfile)
    jktest = jk.SphericalJK( target = CalculateNumDensity, jkargs=[catalog, random], jkargsby=[['RA', 'DEC'],['RA', 'DEC']], \
                            nojkargs=[], regions = jkfile)
    jktest.DoJK( regions = jkfile )
    jkresults = jktest.GetResults(jk=True, full = True)

    
    full_j = jkresults['full']   
    it = jkresults['it']
    it_j = jkresults['jk']

    norm = (njack-1)*1./njack
    #cov = np.std(it_j)**2 * norm
    cov = np.sum((it_j - it_j.mean())**2) * norm
    
    
    os.remove(jkfile)
    return it_j.mean(), it_j, np.sqrt(cov)

In [ ]:
# plot------------------
fig,(ax, ax2, ax3) = plt.subplots(1, 3, figsize = (17, 5))


ax.axhspan( mean_n_density_ngc - std_n_density_ngc, mean_n_density_ngc + std_n_density_ngc, alpha=0.3, color='grey')#, label = r'\rm 68% confidence}')
ax.axhspan( mean_n_density_ngc - 2*std_n_density_ngc, mean_n_density_ngc + 2*std_n_density_ngc, alpha=0.1, color='grey')#, label = r'\rm 68 % confidence}')
#ax.axhspan( 1. - 2*defined_sigma_st82/mean, 1 + 2*defined_sigma_st82/mean, alpha=0.1, color='grey', linestyle = None, label = '95 % confidence')
#ax.errorbar( np.arange(n_density_ngc_norm.size), n_density_ngc_norm, yerr = err_ngc, fmt='o' )
ax.plot( np.arange(avg_n_density_ngc.size), avg_n_density_ngc, marker = '.' , linestyle = 'none', color = 'black' )
ax.plot( avg_n_density_ngc.size/2., ntotal_st82/n_density_ngc.mean(), marker = '*', color = 'red',  markersize = 10, linestyle = 'none', label = r'{\rm train}')
ax.plot( avg_n_density_ngc.size/2., ntotal_DMASS/n_density_ngc.mean(), marker = '*', color = 'blue',  markersize = 10, linestyle = 'none', label = r'{\rm DMASS}')

ax.axhline(y = mean_n_density_ngc, linestyle='--', color = 'grey')
ax.set_title(r'{\rm CMASS NGC}', fontsize = 20)
ax.set_ylim(0.85, 1.15)
ax.legend(loc=4, fontsize = 15)
ax.set_ylabel(r'$n_i / \langle n \rangle$', fontsize = 20)
ax.set_xlabel(r'{\rm patch index $i$}', fontsize = 20)
ax.text(0, 1.12, r'{\rm  Patch area }'+ r'${:0.0f} \deg^2$'.format(area_ngc), fontsize = 15)
#ax.text(0, 1.10, r'{\rm  St82 area }'+ r'${:0.0f} \deg^2$'.format(123), fontsize = 15)
ax.tick_params(labelsize = 15)


ax2.axhspan( mean_n_density_sgc - std_n_density_sgc, mean_n_density_sgc + std_n_density_sgc, alpha=0.3, color='grey')#, label = '68 % confidence')
ax2.axhspan( mean_n_density_sgc - 2*std_n_density_sgc, mean_n_density_sgc + 2*std_n_density_sgc, alpha=0.1, color='grey')#, label = '68 % confidence')
#ax.axhspan( 1. - 2*defined_sigma_st82/mean, 1 + 2*defined_sigma_st82/mean, alpha=0.1, color='grey', linestyle = None, label = '95 % confidence')
#ax.errorbar( np.arange(n_density_ngc_norm.size), n_density_ngc_norm, yerr = err_ngc, fmt='o' )
ax2.plot( np.arange(avg_n_density_sgc.size), avg_n_density_sgc, marker = '.' , linestyle = 'none', color = 'black' )
ax2.plot( avg_n_density_sgc.size/2., ntotal_st82/n_density_sgc.mean(), marker = '*', color = 'red',  markersize = 10, linestyle = 'none', label = r'{\rm TRAIN}')
ax2.plot( avg_n_density_sgc.size/2., ntotal_DMASS/n_density_sgc.mean(), marker = '*', color = 'blue',  markersize = 10, linestyle = 'none', label = r'{\rm DMASS}')
ax2.axhline(y = mean_n_density_sgc, linestyle='--', color = 'grey')

ax2.set_title(r'{\rm CMASS SGC}', fontsize = 20)
ax2.set_ylim(0.85, 1.15)
#ax2.legend(loc=4, fontsize = 15)
ax2.set_ylabel(r'$n_i / \langle n \rangle$', fontsize = 20)
ax2.set_xlabel(r'{\rm patch index $i$}', fontsize = 20)
ax2.text(0, 1.12, r'{\rm  Patch area }'+ r'${:0.0f} \deg^2$'.format(area_sgc), fontsize = 15)
#ax2.text(0, 1.10, r'{\rm  St82 area }'+ r'${:0.0f} \deg^2$'.format(123), fontsize = 15)
ax2.tick_params(labelsize = 15)



ax3.axhspan( mean_n_density_dmass - std_n_density_dmass, mean_n_density_dmass + std_n_density_dmass, alpha=0.3, color='grey', label = '68 % confidence')
ax3.axhspan( mean_n_density_dmass - 2*std_n_density_dmass, mean_n_density_dmass + 2*std_n_density_dmass, alpha=0.1, color='grey', label = '68 % confidence')
#ax.axhspan( 1. - 2*defined_sigma_st82/mean, 1 + 2*defined_sigma_st82/mean, alpha=0.1, color='grey', linestyle = None, label = '95 % confidence')
#ax.errorbar( np.arange(n_density_ngc_norm.size), n_density_ngc_norm, yerr = err_ngc, fmt='o' )
ax3.plot( np.arange(avg_n_density_dmass.size), avg_n_density_dmass, marker = '.' , linestyle = 'none', color = 'black' )
ax3.plot( avg_n_density_dmass.size/2., ntotal_st82/n_density_dmass.mean(), marker = '*', color = 'red',  markersize = 10, linestyle = 'none', label = 'TRAIN')
ax3.axhline(y = mean_n_density_dmass, linestyle='--', color = 'grey')

ax3.set_title(r'{\rm DMASS}', fontsize = 20)
ax3.set_ylim(0.85, 1.15)
#ax3.legend(loc=4, fontsize = 15)
ax3.set_ylabel(r'$n_i / \langle n \rangle$', fontsize = 20)
ax3.set_xlabel(r'{\rm patch index $i$}', fontsize = 20)
ax3.text(0, 1.12, r'{\rm  Patch area }'+ r'${:0.0f} \deg^2$'.format(area_dmass), fontsize = 15)
#ax3.text(0, 1.10, r'{\rm  St82 area }'+ r'${:0.0f} \deg^2$'.format(123), fontsize = 15)
ax3.tick_params(labelsize = 15)

fig.tight_layout()
#fig.savefig('../paper_figure/ndensity-sameA.pdf')